In [1]:
import pandas as pd
import altair as alt

In [5]:
from ecostyles import EcoStyles
# Create styles instance
styles = EcoStyles()
# Register and enable a theme
styles.register_and_enable_theme(theme_name="article")  # or "article"

In [151]:
ind_policy_df = pd.DataFrame({
    'year': [2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023],
    'measure': [34, 20, 56, 144, 136, 242, 260, 228, 705, 632, 852, 1594, 1568, 2049],
    #'act/proj': ['Actual', 'Actual', 'Actual', 'Actual', 'Actual', 'Actual', 'Actual', 'Actual', 'Projected', 'Projected', 'Projected', 'Projected', 'Projected', 'Projected', 'Projected']  
})

In [157]:
base = alt.Chart(ind_policy_df).encode(
    x='year:O',
    y=alt.Y('measure:Q'),
    #color=alt.Color('act/proj:N', legend=alt.Legend(orient='top')),
    tooltip=[
    #alt.Tooltip('act/proj:N', title='Actual/Projected'),
    alt.Tooltip('year:O', title='Year'),
    alt.Tooltip('measure:Q', title='Industrial Policies', format=',.0f')
]
)

line_points = base.mark_line(point=True)

# Custom y-axis label as text, rotated and positioned manually
title_ = alt.Chart(pd.DataFrame({'label': ['Trend of new industrial policy measures, 2010–23']})).mark_text(
    angle=0,
    align='left',
    fontSize=15,
    fontWeight='bold',
    dy=-170,  # vertical offset (adjust as needed)
    dx=-205     # horizontal offset (adjust as needed)
).encode(
    text='label:N'
)

# Layer the fake y-axis label and the bar chart
chart1 = alt.layer(title_, line_points).resolve_scale(
    color='independent'
).configure_axis(
    labelFontSize=12
)

chart1

alt.LayerChart(...)

In [158]:
# Save to png
chart1.save('IPM_chart.png', scale_factor=2)
# Save to json
chart1.save('IPM_chart.json', scale_factor=2)

In [172]:
chart2_data = {
    "Policy instrument": [
        "Financial grant (AE)", "State loan (AE)", "State aid, unespecified (AE)",
        "Public procurement localisation (AE)", "Trade finance (AE)", "Financial assistance in foreign markets (AE)",
        "Controls on transactions and investments (AE)", "Loan guarantee (AE)", "Tax or social insurance relief (AE)",
        "Capital and equity injection (AE)", "Import tariff (EMDE)", "State loan (EMDE)",
        "Tax or social insurance relief (EMDE)", "Capital and quity injection (EMDE)", "State aid, unespecified (EMDE)",
        "Financial grant (EMDE)", "Export tax (EMDE)", "Public procurement localisation (EMDE)",
        "Import licensing requirement (EMDE)", "Internal taxation of imports (EMDE)"
    ],
    "Share in industrial policy (%)": [
        27, 12, 9, 7, 5, 5, 5, 4, 4, 3, 14, 11, 11, 6, 5, 5, 5, 4, 4, 4
    ],
    "Type": [
        "Subsidies", "Subsidies", "Subsidies", "Other calculations", "Subsidies", "Subsidies",
        "Other calculations", "Subsidies", "Subsidies", "Subsidies", "Import/Export measures",
        "Subsidies", "Subsidies", "Subsidies", "Subsidies", "Subsidies", "Import/Export measures",
        "Other calculations", "Import/Export measures", "Import/Export measures"
    ],
    "country": [
        "Advanced economy"] * 10 + ["Emerging market and developing economies"] * 10
}

IP_share_df = pd.DataFrame(chart2_data)


In [176]:
custom_order = [
    'Financial grant (AE)',
    'State loan (AE)',
    'State aid, unespecified (AE)',
    'Public procurement localisation (AE)',
    'Trade finance (AE)',
    'Financial assistance in foreign markets (AE)',
    'Controls on transactions and investments (AE)',
    'Loan guarantee (AE)',
    'Tax or social insurance relief (AE)',
    'Capital and equity injection (AE)',
    'Import tariff (EMDE)',
    'State loan (EMDE)',
    'Tax or social insurance relief (EMDE)',
    'Capital and equity injection (EMDE)',
    'State aid, unespecified (EMDE)',
    'Financial grant (EMDE)',
    'Export tax (EMDE)',
    'Public procurement localisation (EMDE)',
    'Import licensing requirement (EMDE)',
    'Internal taxation of imports (EMDE)'
]

In [179]:
IP_share_df['Policy instrument'] = pd.Categorical(IP_share_df['Policy instrument'], categories=custom_order, ordered=True)

In [180]:
# Step 1: Convert to categorical with custom order
IP_share_df['Policy instrument'] = pd.Categorical(IP_share_df['Policy instrument'], categories=custom_order, ordered=True)

rect_df['y_start'] = pd.Categorical(rect_df['y_start'], categories=custom_order, ordered=True)
rect_df['y_end'] = pd.Categorical(rect_df['y_end'], categories=custom_order, ordered=True)

# Step 2: Build charts without sort, just encode as nominal (categorical)
rects = alt.Chart(rect_df).mark_rect(opacity=0.15).encode(
    y='y_start:N',
    y2='y_end:N',
    color=alt.Color('country:N', scale=alt.Scale(domain=list(group_colors.keys()), range=list(group_colors.values())), legend=None),
    tooltip='country:N'
).properties(width=300, height=450)

bars = alt.Chart(IP_share_df).mark_bar().encode(
    x=alt.X('Share in industrial policy (%)', axis=alt.Axis(format='.0%')),
    y=alt.Y('Policy instrument:N', axis=alt.Axis(labelLimit=400)),
    color=alt.Color('Type:N', legend=alt.Legend(orient='top'))
).properties(height=450, width=300)

title_ = alt.Chart(pd.DataFrame({'label': ['Share in industrial policies (%), by policy instrument, 2023']})).mark_text(
    align='left',
    fontSize=14,
    fontWeight='bold',
    dy=-275,
    dx=-150
).encode(text='label:N')

final_chart = alt.layer(rects, bars, title_).resolve_scale(color='independent').configure_axis(labelFontSize=12)

final_chart

alt.LayerChart(...)

In [178]:
# Create rectangles data for AE and EMDE groups: specify start and end policy instruments per group
rect_data = [
    {
        'country': 'Advanced economy',
        'y_start': 'Financial grant (AE)',
        'y_end': 'Capital and equity injection (AE)'
    },
    {
        'country': 'Emerging market and developing economy',
        'y_start': 'Import tariff (EMDE)',
        'y_end': 'Internal taxation of imports (EMDE)'
    }
]

rect_df = pd.DataFrame(rect_data)

# Define colors for AE and EMDE
group_colors = {
    'Advanced economy': '#a6cee3',
    'Emerging market and developing economy': '#1f78b4'
}

# Background rectangles behind y-axis groups
rects = alt.Chart(rect_df).mark_rect(opacity=0.15).encode(
    y=alt.Y('y_start:N', sort=custom_order),
    y2='y_end:N',
    color=alt.Color('country:N', scale=alt.Scale(domain=list(group_colors.keys()), range=list(group_colors.values())), legend=None),
    tooltip='country:N'
).properties(width=300, height=450)

bars = alt.Chart(IP_share_df).mark_bar().encode(
    x=alt.X('Share in industrial policy (%)', axis=alt.Axis(format='.0%')),
    y=alt.Y('Policy instrument:O', sort=custom_order, axis=alt.Axis(labelLimit=400)),
    color=alt.Color('Type:N', legend=alt.Legend(orient='top'))
).properties(height=450, width=300)

# Custom title text, positioned manually
title_ = alt.Chart(pd.DataFrame({'label': ['Share in industrial policies (%), by policy instrument, 2023']})).mark_text(
    align='left',
    fontSize=14,
    fontWeight='bold',
    dy=-275,
    dx=-150
).encode(text='label:N')

# Layer rectangles behind bars, add title on top
final_chart = alt.layer(rects, bars, title_).resolve_scale(color='independent').configure_axis(labelFontSize=12)

final_chart

alt.LayerChart(...)

In [173]:
# Sort the dataframe by country first, then policy instrument (or whatever order you want)
IP_share_df = IP_share_df.sort_values(by=['country', 'Policy instrument']).reset_index(drop=True)

# Create the sort order list for the y axis (all AE policies first, then EMDE)
y_sort = IP_share_df['Policy instrument'].tolist()

In [174]:
IP_share_df['Share in industrial policy (%)'] = IP_share_df['Share in industrial policy (%)'] / 100

In [82]:
color_mapping = {
    'Subsidies': '#179fdb',
    'Other calculations': '#f4c245',
    'Import/Export measures': '#e6224b'
}

In [175]:
# For each group, get min and max indices in y_sort for rectangle positioning
rect_data = []
for country in IP_share_df['country'].unique():
    group = IP_share_df[IP_share_df['country'] == country]
    first = group.iloc[0]['Policy instrument']
    last = group.iloc[-1]['Policy instrument']
    rect_data.append({'country': country, 'y_start': first, 'y_end': last})

rect_df = pd.DataFrame(rect_data)

# Define colors for the groups
group_colors = {
    'Advanced economy': '#a6cee3',
    'Emerging market and developing economy': '#1f78b4'
}

# Rectangle chart for background highlight
rects = alt.Chart(rect_df).mark_rect(opacity=0.15).encode(
    y=alt.Y('y_start:N', sort=y_sort),
    y2='y_end:N',
    color=alt.Color('country:N', scale=alt.Scale(domain=list(group_colors.keys()), range=list(group_colors.values())), legend=None),
    tooltip='country:N'
).properties(width=300, height=450)

base = alt.Chart(IP_share_df).encode(
    x=alt.X("Share in industrial policy (%)", axis=alt.Axis(format=".0%")),
    y=alt.Y("Policy instrument:N", sort=y_sort, axis=alt.Axis(labelLimit=400)),
    color=alt.Color("Type:N", legend=alt.Legend(orient='top'), scale=alt.Scale(
                        domain=list(color_mapping.keys()),
                        range=list(color_mapping.values())
                    )),
    tooltip=[
        alt.Tooltip('country:N', title='Economy'),
        alt.Tooltip('Policy instrument:N', title='Policy instrument'),
        alt.Tooltip('Share in industrial policy (%)', title='Share in industrial policies', format='.0%')
    ]
).mark_bar().properties(height=450, width=300)

# Custom y-axis label as text, rotated and positioned manually
title_ = alt.Chart(pd.DataFrame({'label': ['Share in industrial policies (%), by policy instrument, 2023']})).mark_text(
    angle=0,
    align='left',
    fontSize=14,
    fontWeight='bold',
    dy=-265,  # vertical offset (adjust as needed)
    dx=-150     # horizontal offset (adjust as needed)
).encode(
    text='label:N'
)


# Layer all charts
chart2 = alt.layer(rects,title_, base).resolve_scale(
    color='independent'
).configure_axis(
    labelFontSize=12
)

chart2


alt.LayerChart(...)

In [143]:
IP_share_df_ae = IP_share_df[IP_share_df['country'] == 'Advanced economy']
#IP_share_df_ae = [x for x in IP_share_df_ae['Policy instrument'].replace(' (AE)','')]

In [146]:
for x in range(len(IP_share_df_ae)):
    IP_share_df_ae.at[x, 'Policy instrument'] = IP_share_df_ae.at[x, 'Policy instrument'].replace(' (AE)', '')    

In [150]:
base = alt.Chart(IP_share_df_ae).encode(
    x=alt.X("Share in industrial policy (%)", axis=alt.Axis(format=".0%")),
    y=alt.Y("Policy instrument:N", sort=alt.EncodingSortField('country'), axis=alt.Axis(labelLimit=400)),
    color=alt.Color("Type:N", legend=alt.Legend(orient='top'), scale=alt.Scale(
                        domain=list(color_mapping.keys()),
                        range=list(color_mapping.values())
                    )),
    tooltip=[
        alt.Tooltip('Policy instrument:N', title='Policy instrument'),
        alt.Tooltip('Share in industrial policy (%)', title='Share in industrial policies', format='.0%')
    ]
).mark_bar().properties(height=450, width=300)

# Custom y-axis label as text, rotated and positioned manually
title_ = alt.Chart(pd.DataFrame({'label': ['Industrial polcies share, by policy instrument, 2023 in Advanced economies']})).mark_text(
    angle=0,
    align='left',
    fontSize=14,
    fontWeight='bold',
    dy=-265,  # vertical offset (adjust as needed)
    dx=-150     # horizontal offset (adjust as needed)
).encode(
    text='label:N'
)


# Layer all charts
chart2_ae = alt.layer(title_, base).resolve_scale(
    color='independent'
).configure_axis(
    labelFontSize=12
)

chart2_ae


alt.LayerChart(...)